In [1]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor


ModuleNotFoundError: No module named 'autogluon'

In [2]:
!pip install autogluon

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     ---------------------------------------- 44.0/44.0 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB ? eta 0:00:00
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
     ---------------------------------------- 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# Cargar datasets
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')
productos_a_predecir = pd.read_csv("datasets/product_id_apredecir201912.csv")

# Merge principal
df = sellin.merge(productos, on="product_id", how="left").merge(stocks, on=["product_id", "periodo"], how="left")
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

# Sumarizar ventas por periodo y producto
ventas = df.groupby(['periodo', 'product_id']).agg({'tn':'sum'}).reset_index()
ventas['periodo_dt'] = pd.to_datetime(ventas['periodo'].astype(str), format='%Y%m')

# Construir toda la historia por producto
vida_producto = ventas.groupby('product_id')['periodo_dt'].agg(['min','max']).reset_index()
all_periods = []
for _, row in vida_producto.iterrows():
    pid = row['product_id']
    periods = pd.date_range(row['min'], row['max'], freq='MS')
    for p in periods:
        all_periods.append((pid, p))
df_full = pd.DataFrame(all_periods, columns=['product_id','periodo_dt'])
ventas = ventas[['product_id','periodo_dt','tn']]
df_pp = df_full.merge(ventas, on=['product_id','periodo_dt'], how='left')
df_pp['tn'] = df_pp['tn'].fillna(0)

# Preparar DataFrame para TimeSeriesDataFrame
df_pp = df_pp.rename(columns={'product_id':'item_id','periodo_dt':'timestamp','tn':'target'})
ts_df = TimeSeriesDataFrame.from_data_frame(df_pp, id_column='item_id', timestamp_column='timestamp')

# Dividir train y test
predict_length = 2  # Predecir 2 meses adelante (enero y febrero 2020)
train_data = ts_df.slice_by_timestep(None, -predict_length)
test_data = ts_df.slice_by_timestep(-predict_length, None)

# Entrenar el modelo
predictor = TimeSeriesPredictor(path='autogluon_timeseries', prediction_length=predict_length, target='target')
predictor.fit(train_data, time_limit=600)


In [ ]:
# Predicciones
productos_ids = productos_a_predecir['product_id'].unique()
test_filtered = test_data[test_data.item_id.isin(productos_ids)]

# Realizar predicciones
forecast = predictor.predict(test_filtered)

# Extraer predicciones para febrero 2020
forecast_df = forecast.to_pandas().reset_index()
forecast_df['timestamp'] = pd.to_datetime(forecast_df['timestamp'])
feb_2020 = pd.Timestamp('2020-02-01')
forecast_feb = forecast_df[forecast_df['timestamp'] == feb_2020][['item_id','mean']].rename(columns={'item_id':'product_id','mean':'tn_pred'})

# Guardar resultados
forecast_feb.to_csv('./kaggle/autogluon_timeseries_feb2020.csv', index=False)
print("✅ Predicciones de febrero 2020 guardadas en ./kaggle/autogluon_timeseries_feb2020.csv")
